In [17]:
import os
import re
import pandas as pd

sql_folder = './sql'  # Folder containing your .sql files
output_folder = './csv'  # Folder to save extracted CSVs
os.makedirs(output_folder, exist_ok=True)

In [18]:
def extract_rows_from_sql(sql_text):
    insert_statements = re.findall(
        r"INSERT INTO [`\"]?([\w_]+)[`\"]?.*?VALUES\s*(.*?);", 
        sql_text, re.DOTALL | re.IGNORECASE
    )
    extracted_data = {}
    for table_name, values_block in insert_statements:
        rows = re.findall(r"\((.*?)\)", values_block, re.DOTALL)
        parsed_rows = []
        for row in rows:
            # Split by comma but ignore commas inside quotes
            values = re.split(r",(?=(?:[^']*'[^']*')*[^']*$)", row)
            cleaned = [v.strip().strip("'").strip('"') for v in values]
            parsed_rows.append(cleaned)
        if table_name in extracted_data:
            extracted_data[table_name].extend(parsed_rows)
        else:
            extracted_data[table_name] = parsed_rows
    return extracted_data

In [19]:
for filename in sorted(os.listdir(sql_folder)):
    if filename.endswith(".sql"):
        path = os.path.join(sql_folder, filename)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        tables = extract_rows_from_sql(content)
        if not tables:
            print(f"⚠️ No valid data found in {filename}")
        for table, rows in tables.items():
            df = pd.DataFrame(rows)
            csv_path = os.path.join(output_folder, f"{table}.csv")
            df.to_csv(csv_path, index=False, header=False)
            print(f"✅ Extracted {len(rows)} rows from {filename} → {table}.csv")

⚠️ No valid data found in 01_mysql_create.sql
✅ Extracted 9235 rows from 02_mysql_populate_author.sql → author.csv
✅ Extracted 568 rows from 03_mysql_populate_publisher.sql → publisher.csv
✅ Extracted 27 rows from 04_mysql_populate_lookups.sql → book_language.csv
✅ Extracted 4 rows from 04_mysql_populate_lookups.sql → shipping_method.csv
✅ Extracted 2 rows from 04_mysql_populate_lookups.sql → address_status.csv
✅ Extracted 6 rows from 04_mysql_populate_lookups.sql → order_status.csv
✅ Extracted 37 rows from 05_mysql_populate_book.sql → book.csv
✅ Extracted 17642 rows from 06_mysql_populate_bookauthor.sql → book_author.csv
✅ Extracted 232 rows from 07_mysql_populate_country.sql → country.csv
✅ Extracted 1000 rows from 08_mysql_populate_address.sql → address.csv
✅ Extracted 2000 rows from 09_mysql_populate_customer.sql → customer.csv
⚠️ No valid data found in 10_mysql_populate_others.sql
⚠️ No valid data found in 11_mysql_populate_order.sql
⚠️ No valid data found in 12_mysql_populate_ord

In [2]:

import os
import re
import pandas as pd
import random
from datetime import datetime, timedelta

sql_folder = './sql'
output_folder = './csv_output'
os.makedirs(output_folder, exist_ok=True)

In [3]:
def extract_rows_from_sql(sql_text):
    insert_statements = re.findall(
        r'INSERT INTO [`"]?([\w_]+)[`"]?.*?VALUES\s*(.*?);',
        sql_text, re.DOTALL | re.IGNORECASE
    )
    extracted_data = {}
    for table_name, values_block in insert_statements:
        rows = re.findall(r"\((.*?)\)", values_block, re.DOTALL)
        parsed_rows = []
        for row in rows:
            values = re.split(r",(?=(?:[^']*'[^']*')*[^']*$)", row)
            cleaned = [v.strip().strip("'").strip('"') for v in values]
            parsed_rows.append(cleaned)
        if table_name in extracted_data:
            extracted_data[table_name].extend(parsed_rows)
        else:
            extracted_data[table_name] = parsed_rows
    return extracted_data

In [4]:

def generate_customer_address():
    customer_ids = list(range(1, 2001))
    address_ids = random.choices(range(1, 1001), k=2000)
    status_ids = [1] * 2000

    customer_ids += random.choices(range(1, 2001), k=750)
    address_ids += random.choices(range(1, 1001), k=750)
    status_ids += [1] * 750

    customer_ids += random.choices(range(1, 2001), k=400)
    address_ids += random.choices(range(1, 1001), k=400)
    status_ids += [2] * 400

    customer_ids += random.choices(range(1, 2001), k=200)
    address_ids += random.choices(range(1, 1001), k=200)
    status_ids += [1] * 200

    df = pd.DataFrame({
        'customer_id': customer_ids,
        'address_id': address_ids,
        'status_id': status_ids
    })
    df.to_csv(f"{output_folder}/customer_address.csv", index=False)
    print("✅ customer_address.csv created")
generate_customer_address()

✅ customer_address.csv created


In [5]:

def generate_cust_order():
    num_orders = 7550
    data = {
        "order_date": [datetime.now() - timedelta(days=random.randint(0, 1095)) for _ in range(num_orders)],
        "customer_id": [random.randint(1, 2000) for _ in range(num_orders)],
        "shipping_method_id": [random.randint(1, 4) for _ in range(num_orders)],
        "dest_address_id": [random.randint(1, 1000) for _ in range(num_orders)],
    }
    df = pd.DataFrame(data)
    df["order_date"] = df["order_date"].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.to_csv(f"{output_folder}/cust_order.csv", index=False)
    print("✅ cust_order.csv created")
generate_cust_order()

✅ cust_order.csv created


In [14]:

def generate_order_line():
    record_count = 4000 + 2000 + 1000 + 300 + 500 + 50
    order_ids = [random.randint(1000, 9999) for _ in range(record_count)]
    book_ids = [random.randint(1, 11126) for _ in range(record_count)]
    prices = [round(random.uniform(0, 20), 2) for _ in range(record_count)]

    df = pd.DataFrame({
        'order_id': order_ids,
        'book_id': book_ids,
        'price': prices
    })
    df.to_csv(f"{output_folder}/order_line.csv", index=False)
    print("✅ order_line.csv created")
generate_order_line()

✅ order_line.csv created


In [15]:

def generate_order_history():
    record_count = 7547 + 6800 + 4000 + 3500 + 300 + 200
    order_ids = list(range(1001, 1001 + record_count))
    status_ids = [1]*7547 + [2]*6800 + [3]*4000 + [4]*3500 + [5]*300 + [6]*200
    random.shuffle(status_ids)
    status_dates = [datetime.now() - timedelta(days=random.randint(1, 30)) for _ in range(len(status_ids))]

    df = pd.DataFrame({
        'order_id': random.choices(order_ids, k=len(status_ids)),
        'status_id': status_ids,
        'status_date': [d.strftime('%Y-%m-%d %H:%M:%S') for d in status_dates]
    })
    df.to_csv(f"{output_folder}/order_history.csv", index=False)
    print("✅ order_history.csv created")
generate_order_history()

✅ order_history.csv created


In [16]:
print('🎉 All SQL files processed and missing ones generated as CSV.')

🎉 All SQL files processed and missing ones generated as CSV.


In [6]:

import pandas as pd
import os

# Load CSVs and check structure
folder_path = '.'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and not f.startswith('cleaned_')]

for file in sorted(files):
    print(f"Loading: {file}")
    df = pd.read_csv(file)
    print(f"\n{'='*40}\nDATAFRAME: {file.upper().replace('.CSV','')}\nShape: {df.shape}")
    print("Missing values:\n", df.isnull().sum())

Loading: address.csv

DATAFRAME: ADDRESS
Shape: (999, 5)
Missing values:
 1                      0
57                     0
Glacier Hill Avenue    0
Torbat-e Jām           0
95                     0
dtype: int64
Loading: address_status.csv

DATAFRAME: ADDRESS_STATUS
Shape: (1, 2)
Missing values:
 1         0
Active    0
dtype: int64
Loading: author.csv

DATAFRAME: AUTHOR
Shape: (9234, 2)
Missing values:
 A. Bartlett Giamatti    0
1                       2
dtype: int64
Loading: book.csv

DATAFRAME: BOOK
Shape: (36, 7)
Missing values:
 The World''s First Love: Mary  Mother of God     0
1                                               10
8987059752                                      10
2                                               10
276                                             10
1996-09-01                                      10
1010                                            10
dtype: int64
Loading: book_author.csv

DATAFRAME: BOOK_AUTHOR
Shape: (17641, 2)
Missing values:
 1570  

In [7]:

files = [f for f in os.listdir('.') if f.endswith('.csv') and not f.startswith('cleaned_')]

for file in sorted(files):
    print(f"Cleaning: {file}")
    df = pd.read_csv(file)
    df_cleaned = df.dropna()
    df_cleaned.to_csv(f"cleaned_{file}", index=False)

print("✅ Cleaned files saved with 'cleaned_' prefix.")

Cleaning: address.csv
Cleaning: address_status.csv
Cleaning: author.csv
Cleaning: book.csv
Cleaning: book_author.csv
Cleaning: book_language.csv
Cleaning: churn_model_data.csv
Cleaning: country.csv
Cleaning: cust_order.csv
Cleaning: customer.csv
Cleaning: customer_address.csv
Cleaning: order_history.csv
Cleaning: order_line.csv
Cleaning: order_status.csv
Cleaning: publisher.csv
Cleaning: shipping_method.csv
✅ Cleaned files saved with 'cleaned_' prefix.


In [8]:
import pandas as pd

# Load cleaned CSV files
cust_order = pd.read_csv("cleaned_cust_order.csv")
customer = pd.read_csv("cleaned_customer.csv")
customer_address = pd.read_csv("cleaned_customer_address.csv")


In [9]:
# --- Feature 1: Order Count per Customer ---
order_count = cust_order['customer_id'].value_counts().rename("order_count")

# --- Feature 2: Shipping Method Variation per Customer ---
shipping_variation = cust_order.groupby('customer_id')['shipping_method_id'].nunique().rename("shipping_variation")

# --- Feature 3: Address Count per Customer ---
address_count = customer_address.groupby('customer_id')['address_id'].nunique().rename("address_count")


In [11]:
print(customer.columns.tolist())

['1', 'Ursola', 'Purdy', 'upurdy0@cdbaby.com']


In [12]:
# Step 1: Load the CSV without assuming the first row is the header
customer = pd.read_csv("cleaned_customer.csv", header=None)

# Step 2: Assign proper column names manually
customer.columns = ['customer_id', 'first_name', 'last_name', 'email']

# Step 3: Confirm it's fixed
print(customer.head())

   customer_id first_name last_name                   email
0            1     Ursola     Purdy      upurdy0@cdbaby.com
1            2   Ruthanne    Vatini       rvatini1@fema.gov
2            3     Reidar   Turbitt  rturbitt2@geocities.jp
3            4       Rich     Kirsz      rkirsz3@jalbum.net
4            5    Carline     Kupis        ckupis4@tamu.edu


In [13]:
# --- Combine All Features ---
features_df = pd.DataFrame({
    'customer_id': customer['customer_id']
}).set_index('customer_id')

features_df = features_df.join([order_count, shipping_variation, address_count])
features_df.fillna(0, inplace=True)
features_df = features_df.astype(int)

# --- Target Label: Churn ---
features_df['churn'] = features_df['order_count'].apply(lambda x: 1 if x < 5 else 0)

features_df.reset_index(inplace=True)


In [14]:
print("✅ Feature engineering completed. Sample output:")
print(features_df.head())

# Optional: Save for next step
features_df.to_csv("features_dataset.csv", index=False)

✅ Feature engineering completed. Sample output:
   customer_id  order_count  shipping_variation  address_count  churn
0            1            4                   3              2      1
1            2            6                   3              1      0
2            3            0                   0              1      1
3            4            3                   2              1      1
4            5            3                   2              2      1


In [18]:
print(df['churn'].value_counts())

churn
1    1355
0     645
Name: count, dtype: int64


In [19]:
print(X.columns)

Index(['order_count', 'shipping_variation', 'address_count'], dtype='object')


In [31]:
import pandas as pd

# Load your feature dataset
df = pd.read_csv("features_dataset.csv")

# Load the order data to calculate recency
cust_order = pd.read_csv("cleaned_cust_order.csv")
cust_order['order_date'] = pd.to_datetime(cust_order['order_date'])

# Get most recent order date per customer
last_order = cust_order.groupby('customer_id')['order_date'].max()

# Calculate days since last order
days_since_last_order = (pd.Timestamp.today() - last_order).dt.days.rename("days_since_last_order")
if 'days_since_last_order' in df.columns:
    df.drop(columns=['days_since_last_order'], inplace=True)
# Merge into your dataset
df = df.set_index('customer_id').join(days_since_last_order).reset_index()

# Fill missing values (for customers with no orders at all)
df['days_since_last_order'].fillna(df['days_since_last_order'].max(), inplace=True)
df['days_since_last_order'] = df['days_since_last_order'].astype(int)

# Save updated feature set
df.to_csv("features_dataset.csv", index=False)
print("✅ 'days_since_last_order' added and saved to features_dataset.csv")

✅ 'days_since_last_order' added and saved to features_dataset.csv


C:\Users\Admin\AppData\Local\Temp\ipykernel_11304\2714666683.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['days_since_last_order'].fillna(df['days_since_last_order'].max(), inplace=True)


In [35]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# Step 1: Load feature dataset
df = pd.read_csv("features_dataset.csv")

# Step 2: Define smart, fuzzy churn logic
def smart_churn(row):
    if row['days_since_last_order'] > 300 and row['order_count'] < 2 and row['shipping_variation'] <= 1:
        return 1
    elif row['days_since_last_order'] > 500 and row['order_count'] <= 1:
        return 1
    elif random.random() < 0.03:
        return 1
    else:
        return 0

df['churn'] = df.apply(smart_churn, axis=1)

# Step 3: Define features (X) and label (y)
X = df[['order_count', 'shipping_variation', 'address_count', 'days_since_last_order']]
y = df['churn']

# Step 4: Train-test split (with stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 5: Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Step 6: Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, zero_division=0),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred)
    }

# Step 7: Display all model scores
results_df = pd.DataFrame(results).T
results_df_sorted = results_df.sort_values(by="F1 Score", ascending=False)
print("📊 Model Evaluation Results:")
display(results_df_sorted)

# Step 8: Auto-select best model using tie-breaker logic
f1_scores = results_df["F1 Score"]
max_f1 = f1_scores.max()
top_models = [name for name, score in f1_scores.items() if score == max_f1]

# Tie-breaker preference
model_priority = ["Gradient Boosting", "Random Forest", "Logistic Regression", "Decision Tree"]
for model_name in model_priority:
    if model_name in top_models:
        best_model_name = model_name
        break

best_model = models[best_model_name]
joblib.dump(best_model, "best_churn_model.pkl")
print(f"✅ Best model '{best_model_name}' saved as 'best_churn_model.pkl'")

# Step 9: Save predictions for entire dataset
df['predicted_churn'] = best_model.predict(X)
df.to_excel("churn_prediction_results.xlsx", index=False)
print("📄 Saved predictions to 'churn_prediction_results.xlsx'")

📊 Model Evaluation Results:


,Accuracy,Precision,Recall,F1 Score
Random Forest,0.9525,0.857143,0.681818,0.759494
Gradient Boosting,0.9425,0.769231,0.681818,0.722892
Decision Tree,0.9325,0.697674,0.681818,0.689655
Logistic Regression,0.9350,0.846154,0.500000,0.628571


✅ Best model 'Random Forest' saved as 'best_churn_model.pkl'
📄 Saved predictions to 'churn_prediction_results.xlsx'
